## New Ver

In [3]:
import time
import requests as rq
import pandas as pd
import datetime as dt
import plotly.express as px
import plotly.graph_objects as go



In [4]:
def fetch_data(date, query_attempt_count=5):
    attempt = 1
    base_url = "https://data.elexon.co.uk/bmrs/api/v1/forecast/indicated/day-ahead/evolution"
    datetime_obj = dt.datetime.strptime(date, '%Y-%m-%d')
    last_day = datetime_obj - dt.timedelta(days=1)
    last_day_str = last_day.strftime('%Y-%m-%d')

    last_two_p = [47, 48]
    settlement_periods_curr = list(range(1, 47))

    params1 = {
        "settlementDate": last_day_str,
        "settlementPeriod": last_two_p,
        "format": "json",
    }

    params2 = {
        "settlementDate": date,
        "settlementPeriod": settlement_periods_curr,
        "format": "json",
    }
    
    r1, r2 = None, None

    while attempt <= query_attempt_count:
        try:
            r1 = rq.get(base_url, params=params1)
            time.sleep(1)
            r2 = rq.get(base_url, params=params2)
            if r1.status_code == 200 and r2.status_code == 200:
                break
        except Exception as e:
            print(f"Attempt {attempt} failed: {e}. Retrying...")
        
        attempt += 1
        if attempt <= query_attempt_count:
            time.sleep(2)

    if r1 is None or r2 is None or r1.status_code != 200 or r2.status_code != 200:
        raise Exception(f"API request failed after {query_attempt_count} attempts")
    
    return r1, r2

def req_to_df(r1, r2):
    data1 = r1.json()
    data2 = r2.json()

    p1 = pd.DataFrame(data1['data'])
    p2 = pd.DataFrame(data2['data'])

    full_df = pd.concat([p1, p2], ignore_index=True)
    
    return full_df

In [5]:
def convert_col_to_cest(df, col_names=["startTime", "publishTime"]):
    df = df.copy()
    for col in col_names:
        df[f"{col}_cest"] = (
            pd.to_datetime(df[col], utc=True)
            .dt.tz_convert("Europe/Berlin")
        )
    return df

def drop_na_get_final(df):
    df_valid = df.dropna(subset=["indicatedImbalance"]).copy()
    df_valid = df_valid.sort_values("publishTime_cest")
    final_df = (
        df_valid
        .groupby(["settlementDate", "settlementPeriod"])
        .tail(1)
        .reset_index(drop=True)
    )
    return final_df

def create_custom_ordering(final_df):
    order = [47, 48] + list(range(1, 47))
    final_df = final_df.copy()
    final_df["settlementPeriod"] = final_df["settlementPeriod"].astype(int)
    final_df["settlementPeriod_str"] = final_df["settlementPeriod"].astype(str)
    order_str = list(map(str, order))
    return final_df, order_str

def imbalance_sign(df, col="indicatedImbalance"):
    df = df.copy()
    df[col + "_sign"] = df[col].apply(
        lambda x: "Positive" if x >= 0 else "Negative"
    )
    return df

In [6]:
def plot(df, order_str):
    df = df.copy()

    # Make sure SPs are ordered correctly: 47, 48, 1..46
    df["settlementPeriod"] = df["settlementPeriod"].astype(int)
    df["settlementPeriod_str"] = df["settlementPeriod"].astype(str)
    order_index = {sp: i for i, sp in enumerate(order_str)}
    df["sp_sort_key"] = df["settlementPeriod_str"].map(order_index)
    df = df.sort_values("sp_sort_key").reset_index(drop=True)

    # Title (same as before)
    latest_publish = df["publishTime_cest"].max()
    main_date = pd.to_datetime(df["settlementDate"]).max()
    date_str = main_date.strftime("%d %b %Y")
    time_str = latest_publish.strftime("%H:%M %Z")
    title = f"Indicated Imbalance per Settlement Period — {date_str}, {time_str}"

    fig = go.Figure()

    # 1) Single continuous line through ALL points (neutral colour)
    fig.add_trace(
        go.Scatter(
            x=df["settlementPeriod_str"],
            y=df["indicatedImbalance"],
            mode="lines",
            name="progression-line",
            line=dict(width=1, color="grey"),
            hoverinfo="skip",  # hover comes from the markers
        )
    )

    # 2) Positive markers
    pos_mask = df["indicatedImbalance"] >= 0
    fig.add_trace(
        go.Scatter(
            x=df.loc[pos_mask, "settlementPeriod_str"],
            y=df.loc[pos_mask, "indicatedImbalance"],
            mode="markers",
            name="Positive",
            marker=dict(size=8, color="green"),  # or ft_green if you prefer
            hovertemplate=(
                "Settlement Period: %{x}<br>"
                "Indicated Imbalance (MW): %{y}<extra></extra>"
            ),
        )
    )

    # 3) Negative markers
    neg_mask = df["indicatedImbalance"] < 0
    fig.add_trace(
        go.Scatter(
            x=df.loc[neg_mask, "settlementPeriod_str"],
            y=df.loc[neg_mask, "indicatedImbalance"],
            mode="markers",
            name="Negative",
            marker=dict(size=8, color="red"),  # or ft_red
            hovertemplate=(
                "Settlement Period: %{x}<br>"
                "Indicated Imbalance (MW): %{y}<extra></extra>"
            ),
        )
    )

    # Layout / axes (keep whatever FT styling you were using)
    fig.update_layout(
        title=title,
        xaxis=dict(
            title="Settlement Period",
            categoryorder="array",
            categoryarray=order_str,
        ),
        yaxis=dict(
            title="Indicated Imbalance (MW)",
            zeroline=True,
            zerolinewidth=1.5,
            zerolinecolor="black",
        ),
        hovermode="x unified",
    )

    # --- saving (same pattern you already had) ---
    try:
        date_val = pd.to_datetime(df["settlementDate"].iloc[0])
        date_str_file = date_val.strftime("%Y-%m-%d")
    except Exception:
        date_str_file = "unknown_date"

    base = f"part1_imbalance_{date_str_file}"

    try:
        fig.write_image(base + ".png")
        print(f"Saved PNG:  {base}.png")
    except Exception as e:
        print(f"FAILED TO SAVE PNG IMAGE ({base}.png): {e}")

    fig.write_html(base + ".html", include_plotlyjs="cdn")
    print(f"Saved HTML: {base}.html")

    fig.show()


In [7]:
def full_run_and_plot(date, do_plot=True):
    
    r1, r2 = fetch_data(date)
    df_raw = req_to_df(r1, r2)
    df_raw = convert_col_to_cest(df_raw)
    final_df = drop_na_get_final(df_raw)
    final_df, order_str = create_custom_ordering(final_df)
    final_df = imbalance_sign(final_df)

    if do_plot:
        plot(final_df, order_str)

    return final_df  


In [8]:
import plotly.graph_objects as go
import pandas as pd

def plot_diff(prev_df, new_df, order_str, title_suffix=""):
    # Plot the difference between previous and new forecast versions
    prev_df = prev_df.copy()
    new_df = new_df.copy()
    
    # Rename for clarity
    prev_df = prev_df.rename(columns={"indicatedImbalance": "indicatedImbalance_prev"})
    new_df = new_df.rename(columns={"indicatedImbalance": "indicatedImbalance_new"})
    
    # Dates present in each snapshot
    prev_dates = prev_df["settlementDate"].unique()
    new_dates = new_df["settlementDate"].unique()
    
    # Decide merge key: same date vs different dates
    if len(prev_dates) == 1 and len(new_dates) == 1 and prev_dates[0] == new_dates[0]:
        merge_on = ["settlementDate", "settlementPeriod"]
        is_same_date = True
    else:
        merge_on = ["settlementPeriod"]
        is_same_date = False
    
    merged = prev_df.merge(
        new_df,
        on=merge_on,
        how="outer",
        suffixes=("_prev", "_new"),
    )
    
    # Ensure types and SP labels
    merged["settlementPeriod"] = merged["settlementPeriod"].astype(int)
    merged["settlementPeriod_str"] = merged["settlementPeriod"].astype(str)
    
    # Sort by custom SP order so lines run 47,48,1..46
    order_index = {sp: i for i, sp in enumerate(order_str)}
    merged["sp_sort_key"] = merged["settlementPeriod_str"].map(order_index)
    merged = merged.sort_values("sp_sort_key").reset_index(drop=True)
    
    # Compute delta and signs
    merged["delta"] = merged["indicatedImbalance_new"] - merged["indicatedImbalance_prev"]
    
    merged["sign_new"] = merged["indicatedImbalance_new"].apply(
        lambda x: "Positive" if pd.notna(x) and x >= 0
        else "Negative" if pd.notna(x)
        else None
    )
    merged["sign_prev"] = merged["indicatedImbalance_prev"].apply(
        lambda x: "Positive" if pd.notna(x) and x >= 0
        else "Negative" if pd.notna(x)
        else None
    )
    
    # Masks for alignment (after sorting)
    prev_mask = merged["indicatedImbalance_prev"].notna()
    new_mask  = merged["indicatedImbalance_new"].notna()
    both_mask = prev_mask & new_mask
    
    fig = go.Figure()
    
    # --------------------------------------------------
    # 1) Thin grey lines connecting SPs within each snapshot
    # --------------------------------------------------
    if prev_mask.any():
        fig.add_trace(go.Scatter(
            x=merged.loc[prev_mask, "settlementPeriod_str"],
            y=merged.loc[prev_mask, "indicatedImbalance_prev"],
            mode="lines",
            name="Previous (line)",
            line=dict(color="lightgrey", width=1),
            showlegend=False,
            hoverinfo="skip",
        ))
    
    if new_mask.any():
        fig.add_trace(go.Scatter(
            x=merged.loc[new_mask, "settlementPeriod_str"],
            y=merged.loc[new_mask, "indicatedImbalance_new"],
            mode="lines",
            name="Latest (line)",
            line=dict(color="grey", width=1),
            showlegend=False,
            hoverinfo="skip",
        ))
    
    # --------------------------------------------------
    # 2) Previous points (faded markers)
    # --------------------------------------------------
    prev_positive_mask = prev_mask & (merged["sign_prev"] == "Positive")
    prev_negative_mask = prev_mask & (merged["sign_prev"] == "Negative")
    
    if prev_positive_mask.any():
        fig.add_trace(go.Scatter(
            x=merged.loc[prev_positive_mask, "settlementPeriod_str"],
            y=merged.loc[prev_positive_mask, "indicatedImbalance_prev"],
            mode="markers",
            name="Previous (Positive)",
            marker=dict(size=8, color="lightgreen", opacity=0.6),
            showlegend=True,
            hovertemplate=(
                "Settlement Period: %{x}<br>"
                "Indicated Imbalance (MW): %{y}"
                "<extra>Previous</extra>"
            ),
        ))
    
    if prev_negative_mask.any():
        fig.add_trace(go.Scatter(
            x=merged.loc[prev_negative_mask, "settlementPeriod_str"],
            y=merged.loc[prev_negative_mask, "indicatedImbalance_prev"],
            mode="markers",
            name="Previous (Negative)",
            marker=dict(size=8, color="lightcoral", opacity=0.6),
            showlegend=True,
            hovertemplate=(
                "Settlement Period: %{x}<br>"
                "Indicated Imbalance (MW): %{y}"
                "<extra>Previous</extra>"
            ),
        ))
    
    # --------------------------------------------------
    # 3) New points (bold markers)
    # --------------------------------------------------
    new_positive_mask = new_mask & (merged["sign_new"] == "Positive")
    new_negative_mask = new_mask & (merged["sign_new"] == "Negative")
    
    if new_positive_mask.any():
        fig.add_trace(go.Scatter(
            x=merged.loc[new_positive_mask, "settlementPeriod_str"],
            y=merged.loc[new_positive_mask, "indicatedImbalance_new"],
            mode="markers",
            name="Latest (Positive)",
            marker=dict(size=12, color="green", opacity=0.9,
                        line=dict(width=1, color="darkgreen")),
            showlegend=True,
            hovertemplate=(
                "Settlement Period: %{x}<br>"
                "Indicated Imbalance (MW): %{y}"
                "<extra>Latest</extra>"
            ),
        ))
    
    if new_negative_mask.any():
        fig.add_trace(go.Scatter(
            x=merged.loc[new_negative_mask, "settlementPeriod_str"],
            y=merged.loc[new_negative_mask, "indicatedImbalance_new"],
            mode="markers",
            name="Latest (Negative)",
            marker=dict(size=12, color="red", opacity=0.9,
                        line=dict(width=1, color="darkred")),
            showlegend=True,
            hovertemplate=(
                "Settlement Period: %{x}<br>"
                "Indicated Imbalance (MW): %{y}"
                "<extra>Latest</extra>"
            ),
        ))
    
    # --------------------------------------------------
    # 4) Vertical lines between old and new for each SP
    # --------------------------------------------------
    if both_mask.any():
        for _, row in merged[both_mask].iterrows():
            if pd.notna(row["indicatedImbalance_prev"]) and pd.notna(row["indicatedImbalance_new"]):
                color = "green" if row["delta"] > 0 else "red"
                fig.add_trace(go.Scatter(
                    x=[row["settlementPeriod_str"], row["settlementPeriod_str"]],
                    y=[row["indicatedImbalance_prev"], row["indicatedImbalance_new"]],
                    mode="lines",
                    line=dict(color=color, width=2),
                    showlegend=False,
                    hoverinfo="skip",
                ))
    
    # --------------------------------------------------
    # 5) Title / layout
    # --------------------------------------------------
    prev_publish = prev_df["publishTime_cest"].max()
    new_publish  = new_df["publishTime_cest"].max()
    
    prev_time_str = prev_publish.strftime("%H:%M %Z")
    new_time_str  = new_publish.strftime("%H:%M %Z")
    
    if is_same_date:
        main_date = pd.to_datetime(prev_dates[0])
        date_str  = main_date.strftime("%d %b %Y")
        base_title = f"Imbalance per Settlement Period {date_str}: {prev_time_str} vs {new_time_str}"
    else:
        prev_date = pd.to_datetime(prev_dates.max())
        new_date  = pd.to_datetime(new_dates.max())
        prev_date_str = prev_date.strftime("%d %b %Y")
        new_date_str  = new_date.strftime("%d %b %Y")
        base_title = (
            f"Imbalance per Settlement Period "
            f"{prev_date_str} {prev_time_str} vs {new_date_str} {new_time_str}"
        )
    
    if title_suffix:
        base_title = f"{base_title} ({title_suffix})"
    
    fig.update_layout(
        title=base_title,
        xaxis=dict(
            categoryorder="array",
            categoryarray=order_str,
            title="Settlement Period",
        ),
        yaxis_title="Indicated Imbalance (MW)",
        hovermode="closest",
        template="plotly_white",
        legend_title_text="Forecast version",
    )
    
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor="LightGray")
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor="LightGray")
    fig.update_yaxes(
        zeroline=True,
        zerolinewidth=1.5,
        zerolinecolor="black",
    )
    
    # Save file names
    try:
        if "settlementDate" in new_df.columns:
            date_val = pd.to_datetime(new_df["settlementDate"].iloc[0])
            date_str_file = date_val.strftime("%Y-%m-%d")
        else:
            date_str_file = "unknown_date"
    except Exception:
        date_str_file = "unknown_date"

    time_str_file = new_publish.strftime("%Y%m%dT%H%M%S")
    base = f"part1_diff_{date_str_file}_{time_str_file}"

    try:
        fig.write_image(base + ".png")
        print(f"Saved PNG:  {base}.png")
    except Exception as e:
        print(f"FAILED TO SAVE PNG IMAGE error: {e}")

    fig.write_html(base + ".html", include_plotlyjs="cdn")
    print(f"Saved HTML: {base}.html")
    
    fig.show()


In [9]:
def countdown_timer(seconds):
    # Countdown Timer to inform user of query attempts
    while seconds:
        mins, secs = divmod(seconds, 60)
        hours, mins = divmod(mins, 60)
        timeformat = f" Next update in: {hours:02d}:{mins:02d}:{secs:02d}"
        print(timeformat, end='\r', flush=True)
        time.sleep(1)
        seconds -= 1
    print(" Checking for new data..." + " " * 30)

In [10]:
def auto_update_loop(
    date,
    update_interval_minutes=30,
    retry=True,
    retry_increments=(30, 60, 120)
):
    # Code to automatically update and plot new data as it becomes available
    print(f" Starting auto-update loop for settlement date: {date}")
    
    # Initial snapshot and plot
    print(" Fetching and plotting initial data...")
    prev_df = full_run_and_plot(date, do_plot=True) 
    prev_df, order_str = create_custom_ordering(prev_df)
    prev_max_publish = prev_df["publishTime_cest"].max()
    print(f" Initial latest publishTime_cest: {prev_max_publish}")
    
    update_cycle = 1
    
    while True:
        # Compute next expected update time based on last publish
        next_expected = prev_max_publish + dt.timedelta(minutes=update_interval_minutes)
        
        # Use same timezone as publishTime_cest
        now = dt.datetime.now(tz=next_expected.tzinfo)
        seconds_to_wait = (next_expected - now).total_seconds()
        
        if seconds_to_wait > 0:
            minutes_to_wait = seconds_to_wait / 60
            print(
                f"\n Update cycle {update_cycle}: "
                f"waiting {minutes_to_wait:.1f} minutes until next expected update at {next_expected}..."
            )
            countdown_timer(int(seconds_to_wait))
        else:
            # if past the expected time -> check immediately
            print(
                f"\n Update cycle {update_cycle}: "
                f"expected update time {next_expected} is already "
                f"{abs(seconds_to_wait) / 60:.1f} minutes in the past, checking now..."
            )
        
        # First attempt to fetch new data
        print(f" Update cycle {update_cycle}: Checking for new data...")
        new_df = full_run_and_plot(date, do_plot=False)
        new_df, _ = create_custom_ordering(new_df)
        new_max_publish = new_df["publishTime_cest"].max()
        
        print(f"Previous publish: {prev_max_publish}")
        print(f"New publish:      {new_max_publish}")
        print(f"Has new data:     {new_max_publish > prev_max_publish}")
        
        if new_max_publish > prev_max_publish:
            print("New data found on first attempt!")
            plot_diff(prev_df, new_df, order_str, title_suffix=f"Update {update_cycle}")
            prev_df = new_df
            prev_max_publish = new_max_publish
            update_cycle += 1
            continue
        
        # 4) Optional retry logic
        if not retry:
            print("No new data, and retry disabled. Loop continues to next interval.")
            update_cycle += 1
            continue
        
        print("No new data on first attempt. Starting retry sequence...")
        retry_found_new_data = False
        
        for inc in retry_increments:
            print(f"Retrying in {inc} seconds...")
            countdown_timer(inc)
            
            new_df = full_run_and_plot(date, do_plot=False)
            new_df, _ = create_custom_ordering(new_df)
            new_max_publish = new_df["publishTime_cest"].max()
            
            print(f"Retry check — new publish: {new_max_publish}")
            
            if new_max_publish > prev_max_publish:
                print(" New data found after retry!")
                plot_diff(prev_df, new_df, order_str, title_suffix=f"Update {update_cycle} (Retry)")
                prev_df = new_df
                prev_max_publish = new_max_publish
                retry_found_new_data = True
                break
        
        if not retry_found_new_data:
            print(" No new data after all retries. Waiting until next expected interval.")
        
        update_cycle += 1


In [72]:
auto_update_loop(
    date="2025-12-07",              # pick your BMRS day here (e.g. between 2024-11-10 and 2024-11-21)
    update_interval_minutes=30,
    retry=True,
    retry_increments=(30, 60, 120),   # 30s, 60s, 120s
)

 Starting auto-update loop for settlement date: 2025-12-07
 Fetching and plotting initial data...
Saved PNG:  part1_imbalance_2025-12-06.png
Saved HTML: part1_imbalance_2025-12-06.html


 Initial latest publishTime_cest: 2025-12-07 20:17:00+01:00

 Update cycle 1: waiting 9.8 minutes until next expected update at 2025-12-07 20:47:00+01:00...
 Checking for new data...                              
 Update cycle 1: Checking for new data...
Previous publish: 2025-12-07 20:17:00+01:00
New publish:      2025-12-07 20:17:00+01:00
Has new data:     False
No new data on first attempt. Starting retry sequence...
Retrying in 30 seconds...
 Checking for new data...                              
Retry check — new publish: 2025-12-07 20:17:00+01:00
Retrying in 60 seconds...
 Checking for new data...                              
Retry check — new publish: 2025-12-07 20:17:00+01:00
Retrying in 120 seconds...
 Checking for new data...                              
Retry check — new publish: 2025-12-07 20:47:00+01:00
 New data found after retry!
Saved PNG:  part1_diff_2025-12-06_20251207T204700.png
Saved HTML: part1_diff_2025-12-06_20251207T204700.html



 Update cycle 2: waiting 20.1 minutes until next expected update at 2025-12-07 21:17:00+01:00...
 Checking for new data...                              
 Update cycle 2: Checking for new data...
Previous publish: 2025-12-07 20:47:00+01:00
New publish:      2025-12-07 21:17:00+01:00
Has new data:     True
New data found on first attempt!
Saved PNG:  part1_diff_2025-12-06_20251207T211700.png
Saved HTML: part1_diff_2025-12-06_20251207T211700.html



 Update cycle 3: waiting 14.3 minutes until next expected update at 2025-12-07 21:47:00+01:00...
 Checking for new data...                              
 Update cycle 3: Checking for new data...
Previous publish: 2025-12-07 21:17:00+01:00
New publish:      2025-12-07 22:47:00+01:00
Has new data:     True
New data found on first attempt!
Saved PNG:  part1_diff_2025-12-06_20251207T224700.png
Saved HTML: part1_diff_2025-12-06_20251207T224700.html



 Update cycle 4: waiting 23.5 minutes until next expected update at 2025-12-07 23:17:00+01:00...
 Checking for new data...                              
 Update cycle 4: Checking for new data...
Previous publish: 2025-12-07 22:47:00+01:00
New publish:      2025-12-07 22:47:00+01:00
Has new data:     False
No new data on first attempt. Starting retry sequence...
Retrying in 30 seconds...
 Checking for new data...                              
Retry check — new publish: 2025-12-07 22:47:00+01:00
Retrying in 60 seconds...
 Checking for new data...                              
Retry check — new publish: 2025-12-07 22:47:00+01:00
Retrying in 120 seconds...
 Checking for new data...                              
Retry check — new publish: 2025-12-07 22:47:00+01:00
 No new data after all retries. Waiting until next expected interval.

 Update cycle 5: expected update time 2025-12-07 23:17:00+01:00 is already 3.8 minutes in the past, checking now...
 Update cycle 5: Checking for new data...



 Update cycle 6: waiting 24.6 minutes until next expected update at 2025-12-07 23:47:00+01:00...
 Checking for new data...                              
 Update cycle 6: Checking for new data...
Previous publish: 2025-12-07 23:17:00+01:00
New publish:      2025-12-07 23:17:00+01:00
Has new data:     False
No new data on first attempt. Starting retry sequence...
Retrying in 30 seconds...
 Checking for new data...                              
Retry check — new publish: 2025-12-07 23:17:00+01:00
Retrying in 60 seconds...
 Checking for new data...                              
Retry check — new publish: 2025-12-07 23:17:00+01:00
Retrying in 120 seconds...
 Checking for new data...                              
Retry check — new publish: 2025-12-07 23:17:00+01:00
 No new data after all retries. Waiting until next expected interval.

 Update cycle 7: expected update time 2025-12-07 23:47:00+01:00 is already 52.2 minutes in the past, checking now...
 Update cycle 7: Checking for new data...

KeyboardInterrupt: 

In [11]:
def test_plot_diff_different_dates():
    # FOR TEST PURPOSES ONLY
    print("Testing with different dates...")
    
    # Choose two dates that will definitely have different data
    date1 = "2025-11-10"
    date2 = "2025-11-12"
    
    # Get data for both dates
    df1 = full_run_and_plot(date1, do_plot=False)
    df2 = full_run_and_plot(date2, do_plot=False)
    
    # For testing only: create a modified version of plot_diff that ignores settlementDate
    def test_plot_diff_version(prev_df, new_df, order_str):
        """Modified plot_diff that merges only on settlementPeriod for testing"""
        # Work on copies
        prev_df = prev_df.copy()
        new_df = new_df.copy()
        
        # Rename for clarity
        prev_df = prev_df.rename(columns={"indicatedImbalance": "indicatedImbalance_prev"})
        new_df = new_df.rename(columns={"indicatedImbalance": "indicatedImbalance_new"})
        
        # Merge only on settlementPeriod for testing (different dates)
        merged = prev_df.merge(
            new_df,
            on=["settlementPeriod"],
            how="outer",
            suffixes=("_prev", "_new")
        )
        
        # Ensure proper types and ordering
        merged["settlementPeriod"] = merged["settlementPeriod"].astype(int)
        merged["settlementPeriod_str"] = merged["settlementPeriod"].astype(str)
        
        # Compute delta and signs
        merged["delta"] = merged["indicatedImbalance_new"] - merged["indicatedImbalance_prev"]
        
        merged["sign_new"] = merged["indicatedImbalance_new"].apply(
            lambda x: "Positive" if pd.notna(x) and x >= 0 else "Negative" if pd.notna(x) else None
        )
        
        merged["sign_prev"] = merged["indicatedImbalance_prev"].apply(
            lambda x: "Positive" if pd.notna(x) and x >= 0 else "Negative" if pd.notna(x) else None
        )
        
        # Masks for plotting
        prev_mask = merged["indicatedImbalance_prev"].notna()
        new_mask = merged["indicatedImbalance_new"].notna()
        both_mask = prev_mask & new_mask
        
        fig = go.Figure()
        
        # 1) Previous points (very light colors with opacity)
        prev_positive_mask = prev_mask & (merged["sign_prev"] == "Positive")
        prev_negative_mask = prev_mask & (merged["sign_prev"] == "Negative")
        
        # Previous positive points (very light green)
        if prev_positive_mask.any():
            fig.add_trace(go.Scatter(
                x=merged.loc[prev_positive_mask, "settlementPeriod_str"],
                y=merged.loc[prev_positive_mask, "indicatedImbalance_prev"],
                mode="markers",
                name="Previous (Positive)",
                marker=dict(size=8, color="lightgreen", opacity=0.6),
                showlegend=True
            ))
        
        # Previous negative points (very light red)
        if prev_negative_mask.any():
            fig.add_trace(go.Scatter(
                x=merged.loc[prev_negative_mask, "settlementPeriod_str"],
                y=merged.loc[prev_negative_mask, "indicatedImbalance_prev"],
                mode="markers",
                name="Previous (Negative)",
                marker=dict(size=8, color="lightcoral", opacity=0.6),
                showlegend=True
            ))
        
        # 2) New points (bold colors)
        new_positive_mask = new_mask & (merged["sign_new"] == "Positive")
        new_negative_mask = new_mask & (merged["sign_new"] == "Negative")
        
        # New positive points (bold green)
        if new_positive_mask.any():
            fig.add_trace(go.Scatter(
                x=merged.loc[new_positive_mask, "settlementPeriod_str"],
                y=merged.loc[new_positive_mask, "indicatedImbalance_new"],
                mode="markers",
                name="Latest (Positive)",
                marker=dict(size=12, color="green", opacity=0.9, line=dict(width=1, color="darkgreen")),
                showlegend=True
            ))
        
        # New negative points (bold red)
        if new_negative_mask.any():
            fig.add_trace(go.Scatter(
                x=merged.loc[new_negative_mask, "settlementPeriod_str"],
                y=merged.loc[new_negative_mask, "indicatedImbalance_new"],
                mode="markers",
                name="Latest (Negative)",
                marker=dict(size=12, color="red", opacity=0.9, line=dict(width=1, color="darkred")),
                showlegend=True
            ))
        
        # 3) Lines between previous and new (colored by change direction)
        # Only draw lines where we have both previous and new values
        if both_mask.any():
            for idx, row in merged[both_mask].iterrows():
                if pd.notna(row["indicatedImbalance_prev"]) and pd.notna(row["indicatedImbalance_new"]):
                    # Green line if value increased, red if decreased
                    color = "#4CAF50" if row["delta"] > 0 else "#E74C3C"
                    
                    fig.add_trace(go.Scatter(
                        x=[row["settlementPeriod_str"], row["settlementPeriod_str"]],
                        y=[row["indicatedImbalance_prev"], row["indicatedImbalance_new"]],
                        mode="lines",
                        line=dict(color=color, width=2, dash="dot"),
                        showlegend=False,
                        hoverinfo="skip"
                    ))
        
        fig.update_layout(
            title="Latest vs Previous Forecast — Auto-Update Diff (TEST MODE: Different Dates)",
            xaxis=dict(
                categoryorder="array", 
                categoryarray=order_str,
                title="Settlement Period"
            ),
            yaxis_title="Indicated Imbalance (MW)",
            hovermode="x unified",
            template="plotly_white"
        )
        
        # Add grid lines
        fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')
        fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')
        fig.update_yaxes(
            zeroline=True,
            zerolinewidth=2.5,     # thickness
            zerolinecolor="black" # your FT axis colour
        )

        # Show some debug info
        print(f"   Data summary:")
        print(f"   Previous data points: {prev_mask.sum()}")
        print(f"   New data points: {new_mask.sum()}")
        print(f"   Overlapping periods (with lines): {both_mask.sum()}")
        
        fig.show()
    
    # Call the test version
    df1, order_str = create_custom_ordering(df1)
    print(f"Testing {date1} (prev) vs {date2} (new)")
    test_plot_diff_version(df1, df2, order_str)

In [ ]:
def test_plot_diff_different_dates(use_ft_style=False):
    """
    Quick visual test for the plot_diff hover behaviour.

    Shows two figures:
      1) hovermode='x unified'  -> old, buggy behaviour (multiple entries in tooltip)
      2) hovermode='closest'    -> new, fixed behaviour (single point in tooltip)

    Parameters
    ----------
    use_ft_style : bool
        If True, use FT-style background, fonts and grid.
        If False, use default Plotly white template.
    """
    print("Testing plot_diff hover behaviour with different dates...")

    # Choose two dates that will definitely have different data
    date1 = "2025-11-10"
    date2 = "2025-11-12"

    # Get data for both dates (no plotting here)
    df1 = full_run_and_plot(date1, do_plot=False)
    df2 = full_run_and_plot(date2, do_plot=False)

    # Build the custom settlementPeriod ordering from df1
    df1, order_str = create_custom_ordering(df1)

    def test_plot_diff_version(prev_df, new_df, order_str,
                               hovermode="closest",
                               title_suffix="",
                               use_ft_style=False):
        """Test-only version of plot_diff, merging only on settlementPeriod."""
        prev_df = prev_df.copy()
        new_df = new_df.copy()

        # Rename for clarity
        prev_df = prev_df.rename(columns={"indicatedImbalance": "indicatedImbalance_prev"})
        new_df = new_df.rename(columns={"indicatedImbalance": "indicatedImbalance_new"})

        # Merge only on settlementPeriod (ignore settlementDate on purpose)
        merged = prev_df.merge(
            new_df,
            on=["settlementPeriod"],
            how="outer",
            suffixes=("_prev", "_new"),
        )

        # Types / labels
        merged["settlementPeriod"] = merged["settlementPeriod"].astype(int)
        merged["settlementPeriod_str"] = merged["settlementPeriod"].astype(str)

        # Delta and signs
        merged["delta"] = merged["indicatedImbalance_new"] - merged["indicatedImbalance_prev"]

        merged["sign_new"] = merged["indicatedImbalance_new"].apply(
            lambda x: "Positive" if pd.notna(x) and x >= 0
            else "Negative" if pd.notna(x)
            else None
        )
        merged["sign_prev"] = merged["indicatedImbalance_prev"].apply(
            lambda x: "Positive" if pd.notna(x) and x >= 0
            else "Negative" if pd.notna(x)
            else None
        )

        # Masks
        prev_mask = merged["indicatedImbalance_prev"].notna()
        new_mask = merged["indicatedImbalance_new"].notna()
        both_mask = prev_mask & new_mask

        fig = go.Figure()

        # Previous points (faded)
        prev_positive_mask = prev_mask & (merged["sign_prev"] == "Positive")
        prev_negative_mask = prev_mask & (merged["sign_prev"] == "Negative")

        if prev_positive_mask.any():
            fig.add_trace(go.Scatter(
                x=merged.loc[prev_positive_mask, "settlementPeriod_str"],
                y=merged.loc[prev_positive_mask, "indicatedImbalance_prev"],
                mode="markers",
                name="Previous (Positive)",
                marker=dict(size=8, color="lightgreen", opacity=0.6),
                hovertemplate=(
                    "Previous<br>SP: %{x}<br>"
                    "Imbalance: %{y:.1f} MW<extra></extra>"
                ),
            ))

        if prev_negative_mask.any():
            fig.add_trace(go.Scatter(
                x=merged.loc[prev_negative_mask, "settlementPeriod_str"],
                y=merged.loc[prev_negative_mask, "indicatedImbalance_prev"],
                mode="markers",
                name="Previous (Negative)",
                marker=dict(size=8, color="lightcoral", opacity=0.6),
                hovertemplate=(
                    "Previous<br>SP: %{x}<br>"
                    "Imbalance: %{y:.1f} MW<extra></extra>"
                ),
            ))

        # New points (bold)
        new_positive_mask = new_mask & (merged["sign_new"] == "Positive")
        new_negative_mask = new_mask & (merged["sign_new"] == "Negative")

        if new_positive_mask.any():
            fig.add_trace(go.Scatter(
                x=merged.loc[new_positive_mask, "settlementPeriod_str"],
                y=merged.loc[new_positive_mask, "indicatedImbalance_new"],
                mode="markers",
                name="Latest (Positive)",
                marker=dict(size=12, color="green", opacity=0.9,
                            line=dict(width=1, color="darkgreen")),
                hovertemplate=(
                    "Latest<br>SP: %{x}<br>"
                    "Imbalance: %{y:.1f} MW<extra></extra>"
                ),
            ))

        if new_negative_mask.any():
            fig.add_trace(go.Scatter(
                x=merged.loc[new_negative_mask, "settlementPeriod_str"],
                y=merged.loc[new_negative_mask, "indicatedImbalance_new"],
                mode="markers",
                name="Latest (Negative)",
                marker=dict(size=12, color="red", opacity=0.9,
                            line=dict(width=1, color="darkred")),
                hovertemplate=(
                    "Latest<br>SP: %{x}<br>"
                    "Imbalance: %{y:.1f} MW<extra></extra>"
                ),
            ))

        # Lines between previous and new (only where both exist)
        if both_mask.any():
            for _, row in merged[both_mask].iterrows():
                if pd.notna(row["indicatedImbalance_prev"]) and pd.notna(row["indicatedImbalance_new"]):
                    color = "#4CAF50" if row["delta"] > 0 else "#E74C3C"
                    fig.add_trace(go.Scatter(
                        x=[row["settlementPeriod_str"], row["settlementPeriod_str"]],
                        y=[row["indicatedImbalance_prev"], row["indicatedImbalance_new"]],
                        mode="lines",
                        line=dict(color=color, width=2, dash="dot"),
                        showlegend=False,
                        hoverinfo="skip",
                    ))

        base_title = f"Latest vs Previous Forecast — {title_suffix} (hovermode={hovermode})"

        # ---- layout: plain vs FT-style ----
        if use_ft_style:
            fig.update_layout(
                title=base_title,
                paper_bgcolor=paper_bg,
                plot_bgcolor=plot_bg,
                font=dict(family="Georgia, serif", color=tick_col),
                hovermode=hovermode,
                legend_title_text="Forecast version",
            )
            fig.update_xaxes(
                showgrid=True,
                gridwidth=1,
                gridcolor=grid_col,
                linecolor=axis_col,
                tickfont=dict(color=tick_col),
                categoryorder="array",
                categoryarray=order_str,
                title="Settlement Period",
            )
            fig.update_yaxes(
                showgrid=True,
                gridwidth=1,
                gridcolor=grid_col,
                zeroline=True,
                zerolinewidth=2.0,
                zerolinecolor=axis_col,
                linecolor=axis_col,
                tickfont=dict(color=tick_col),
                title_text="Indicated Imbalance (MW)",
            )
        else:
            fig.update_layout(
                title=base_title,
                hovermode=hovermode,
                template="plotly_white",
            )
            fig.update_xaxes(
                showgrid=True,
                gridwidth=1,
                gridcolor="LightGray",
                categoryorder="array",
                categoryarray=order_str,
                title="Settlement Period",
            )
            fig.update_yaxes(
                showgrid=True,
                gridwidth=1,
                gridcolor="LightGray",
                zeroline=True,
                zerolinewidth=2.0,
                zerolinecolor="black",
                title_text="Indicated Imbalance (MW)",
            )

        print("   Data summary:")
        print(f"   Previous data points: {prev_mask.sum()}")
        print(f"   New data points:      {new_mask.sum()}")
        print(f"   Overlapping periods:  {both_mask.sum()}")

        fig.show()
        return fig

    # --- Run the two comparison plots ---

    print("Plot 1: hovermode='x unified' (OLD behaviour).")
    test_plot_diff_version(
        df1, df2, order_str,
        hovermode="x unified",
        title_suffix=f"{date1} vs {date2} – old",
        use_ft_style=use_ft_style,
    )

    print("Plot 2: hovermode='closest' (FIXED behaviour).")
    fg = test_plot_diff_version(
        df1, df2, order_str,
        hovermode="closest",
        title_suffix=f"{date1} vs {date2} – fixed",
        use_ft_style=use_ft_style,
    )
    fg.show()
    fg.write_image("example.png", width=1600, height = 900, scale = 2)
    fg.write_html("example.html", include_plotlyjs="cdn")

In [17]:
test_plot_diff_different_dates(use_ft_style=True)

Testing plot_diff hover behaviour with different dates...
Plot 1: hovermode='x unified' (OLD behaviour).
   Data summary:
   Previous data points: 48
   New data points:      48
   Overlapping periods:  48


Plot 2: hovermode='closest' (FIXED behaviour).
   Data summary:
   Previous data points: 48
   New data points:      48
   Overlapping periods:  48


TypeError: write_image() got an unexpected keyword argument 'include_plotlyjs'

### FT STYLE

In [14]:
import plotly.express as px

# FT-ish palette
paper_bg = "#f2e6d8"      # warm FT paper
plot_bg  = "#f2e6d8"      # same as paper for seamless look
grid_col = "#e3d5c6"      # very soft grid
axis_col = "#b0977b"      # axis line colour
tick_col = "#6b5a4b"      # text colour

ft_green = "#7bb274"      # muted soft green
ft_red   = "#c6665c"      # muted salmon red

# Make sure imbalanceSign matches map keys
final_df["imbalanceSign"] = final_df["indicatedImbalance"].apply(
    lambda x: "Positive" if x > 0 else "Negative"
)

fig = px.scatter(
    final_df,
    title="Indicated Imbalance per Settlement Period (Final Forecast)",
    x="settlementPeriod_str",
    y="indicatedImbalance",
    color="imbalanceSign",
    color_discrete_map={
        "Positive": ft_green,
        "Negative": ft_red,
    },
    category_orders={"settlementPeriod_str": order_str},
)

# Softer, FT-like markers
fig.update_traces(
    marker=dict(
        size=8,              # slightly smaller, like the example
        opacity=0.9,
        line=dict(width=0)   # no strong outline
    )
)

# Background
fig.update_layout(
    paper_bgcolor=paper_bg,
    plot_bgcolor=plot_bg,
    font=dict(family="Georgia, serif", color=tick_col),
)

# Axes & grid – light and minimal
fig.update_xaxes(
    showgrid=False,                # no vertical grid, like FT example
    linecolor=axis_col,
    tickfont=dict(color=tick_col),
)

fig.update_yaxes(
    gridcolor=grid_col,            # soft horizontal grid
    zerolinecolor=axis_col,        # stronger 0 line
    linecolor=axis_col,
    tickfont=dict(color=tick_col),
)

fig.show()



NameError: name 'final_df' is not defined

# Second Task

In [ ]:
import requests as rq
import pandas as pd
import datetime as dt
import time
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime


In [ ]:
from plotly.subplots import make_subplots


In [ ]:
def fetch_wind_solar_forecast(date, query_attempt_count=5):
    """
    Fetch day-ahead forecast generation for wind & solar (DGWS / B1440)
    for a single UTC day.

    Uses:
      GET /forecast/generation/wind-and-solar/day-ahead
      filtered by startTime via 'from' and 'to'.

    Parameters
    ----------
    date : str
        Settlement date in 'YYYY-MM-DD' (UTC).
        Query attempt: how many times to retry on failure.
    """
    base_url = "https://data.elexon.co.uk/bmrs/api/v1/forecast/generation/wind-and-solar/day-ahead"

    start_iso = f"{date}T00:00Z"
    end_iso = f"{date}T23:30Z"

    params = {
        "from": start_iso,
        "to": end_iso,
        "processType": "Day ahead",
        "format": "json",
    }

    attempt = 1
    r = None

    while attempt <= query_attempt_count:
        try:
            print(f" Forecast attempt {attempt} ...")
            r = rq.get(base_url, params=params)

            if r.status_code == 200:
                print("Forecast request OK.")
                break
            else:
                print(f"Forecast HTTP status: {r.status_code}")
        except Exception as e:
            print(f"Forecast attempt {attempt} failed: {e}")

        attempt += 1
        if attempt <= query_attempt_count:
            time.sleep(2)

    if r is None or r.status_code != 200:
        raise Exception(f"Forecast API request failed after {query_attempt_count} attempts")

    return r


def fetch_wind_solar_actuals(date, query_attempt_count=5):
    """
    Fetch actual/estimated wind & solar generation (AGWS / B1630)
    for a single UTC day.

    Uses:
      GET /generation/actual/per-type/wind-and-solar

    Parameters
    ----------
    date : str
        Settlement date in 'YYYY-MM-DD' (UTC).
        Query attempt: how many times to retry on failure.
    """
    base_url = "https://data.elexon.co.uk/bmrs/api/v1/generation/actual/per-type/wind-and-solar"

    date_obj = dt.datetime.strptime(date, "%Y-%m-%d")
    next_day = date_obj + dt.timedelta(days=1)

    start_iso = date_obj.strftime("%Y-%m-%dT00:00Z")
    end_iso = next_day.strftime("%Y-%m-%dT00:00Z")

    params = {
        "from": start_iso,
        "to": end_iso,
        "settlementPeriodFrom": 1,
        "settlementPeriodTo": 48,
        "format": "json",
    }

    attempt = 1
    r = None

    while attempt <= query_attempt_count:
        try:
            print(f" Actuals attempt {attempt} ...")
            r = rq.get(base_url, params=params)

            if r.status_code == 200:
                print(" Actuals request OK.")
                break
            else:
                print(f"Actuals HTTP status: {r.status_code}")
        except Exception as e:
            print(f"Actuals attempt {attempt} failed: {e}")

        attempt += 1
        if attempt <= query_attempt_count:
            time.sleep(2)

    if r is None or r.status_code != 200:
        raise Exception(f"Actuals API request failed after {query_attempt_count} attempts")

    return r


def forecast_req_to_df(r):
    """
    Convert forecast JSON response to a DataFrame.
    """
    data = r.json()
    return pd.DataFrame(data["data"])


def actuals_req_to_df(r):
    """
    Convert actuals JSON response to a DataFrame.
    """
    data = r.json()
    return pd.DataFrame(data["data"])


In [ ]:
t1 = fetch_wind_solar_forecast("2025-11-15")
t2 = fetch_wind_solar_actuals("2025-11-15")
d1 = forecast_req_to_df(t1)
d2 = forecast_req_to_df(t2)
d1.columns

 Forecast attempt 1 ...
Forecast request OK.
 Actuals attempt 1 ...
 Actuals request OK.


Index(['publishTime', 'processType', 'businessType', 'psrType', 'startTime',
       'settlementDate', 'settlementPeriod', 'quantity'],
      dtype='object')

In [ ]:
d1 = forecast_req_to_df(t1)
d2 = forecast_req_to_df(t2)

In [ ]:
d2.columns

Index(['publishTime', 'businessType', 'psrType', 'quantity', 'startTime',
       'settlementDate', 'settlementPeriod'],
      dtype='object')

In [ ]:
def settlement_period_order():
    """
    BMRS-style ordering for a local day: 47, 48, 1..46.
    """
    return [str(sp) for sp in ([47, 48] + list(range(1, 47)))]



def normalise_mw_column(df, new_col_name):
    """
    Rename the numeric MW column to a unified name.
    Tries common candidates used in EMFIP/BMRS streams:
      - 'quantity'
      - 'generation'
      - 'value'
    """
    df = df.copy()
    candidates = ["quantity", "generation", "value"]

    src_col = None
    for c in candidates:
        if c in df.columns:
            src_col = c
            break

    if src_col is None:
        raise KeyError(
            f"Could not find an MW column in df; "
            f"looked for {candidates}, got: {list(df.columns)}"
        )

    if src_col != new_col_name:
        df = df.rename(columns={src_col: new_col_name})

    return df

def map_psr_to_fuel(psr):
    """
    Map psrType to a simple fuel label.
    """
    if psr is None:
        return None

    psr_lower = str(psr).lower()

    if "solar" in psr_lower:
        return "Solar"
    if "wind" in psr_lower:
        return "Wind"

    return None


def add_fuel_column(df):
    """
    Add a 'fuel' column (Wind / Solar) based on 'psrType'.
    """
    if df is None:
        raise ValueError("DataFrame is None in add_fuel_column().")

    df = df.copy()

    if "psrType" not in df.columns:
        raise KeyError(f"Expected 'psrType' column, got: {list(df.columns)}")

    df["fuel"] = df["psrType"].apply(map_psr_to_fuel)
    df = df[df["fuel"].notna()].reset_index(drop=True)

    return df



In [ ]:
def prepare_wind_solar_merged(forecast_df, actuals_df):
    """
    Align forecast vs actual data.

    - Add:
        forecast_MW = quantity (forecast)
        actual_MW   = quantity (actuals)
        startTime_cest via convert_col_to_cest
        fuel (Wind / Solar)
        diff_MW = actual_MW - forecast_MW
    - Aggregate per (settlementDate, settlementPeriod, fuel).
    """
    forecast_df = forecast_df.copy()
    actuals_df = actuals_df.copy()

    if "quantity" not in forecast_df.columns:
        raise KeyError(f"Forecast DF missing 'quantity'; columns: {list(forecast_df.columns)}")
    if "quantity" not in actuals_df.columns:
        raise KeyError(f"Actuals DF missing 'quantity'; columns: {list(actuals_df.columns)}")

    forecast_df["forecast_MW"] = forecast_df["quantity"]
    actuals_df["actual_MW"] = actuals_df["quantity"]

    forecast_df = convert_col_to_cest(forecast_df, col_names=("startTime",))
    actuals_df = convert_col_to_cest(actuals_df, col_names=("startTime",))

    if "settlementPeriod" in forecast_df.columns:
        forecast_df["settlementPeriod"] = forecast_df["settlementPeriod"].astype(int)
    if "settlementPeriod" in actuals_df.columns:
        actuals_df["settlementPeriod"] = actuals_df["settlementPeriod"].astype(int)

    forecast_df = add_fuel_column(forecast_df)
    actuals_df = add_fuel_column(actuals_df)

    group_cols = ["settlementDate", "settlementPeriod", "fuel"]

    missing_forecast = [c for c in group_cols if c not in forecast_df.columns]
    missing_actual = [c for c in group_cols if c not in actuals_df.columns]

    if missing_forecast:
        raise KeyError(f"Forecast DF missing {missing_forecast}")
    if missing_actual:
        raise KeyError(f"Actuals DF missing {missing_actual}")

    forecast_agg = (
        forecast_df
        .groupby(group_cols, as_index=False)
        .agg({
            "forecast_MW": "sum",
            "startTime_cest": "min",
        })
    )

    actuals_agg = (
        actuals_df
        .groupby(group_cols, as_index=False)
        .agg({
            "actual_MW": "sum",
            "startTime_cest": "min",
        })
    )

    merged = forecast_agg.merge(
        actuals_agg,
        on=group_cols,
        how="inner",
        suffixes=("_forecast", "_actual"),
    )

    merged["startTime_cest"] = merged["startTime_cest_forecast"].combine_first(
        merged["startTime_cest_actual"]
    )

    merged["diff_MW"] = merged["actual_MW"] - merged["forecast_MW"]

    merged = merged.sort_values(["settlementDate", "fuel", "settlementPeriod"]).reset_index(drop=True)

    return merged


In [ ]:
def split_wind_solar(merged_df):
    """
    Split merged_df into separate Wind and Solar DataFrames.
    """
    merged_df = merged_df.copy()

    if "fuel" not in merged_df.columns:
        raise KeyError("'fuel' column not found in merged_df")

    df_wind = merged_df[merged_df["fuel"] == "Wind"].copy()
    df_solar = merged_df[merged_df["fuel"] == "Solar"].copy()

    for df in (df_wind, df_solar):
        if not df.empty:
            df["settlementPeriod"] = df["settlementPeriod"].astype(int)

    return df_wind, df_solar


In [ ]:
from plotly.subplots import make_subplots

def plot_forecast_vs_actual_with_table(df, fuel_label="Wind", x_axis="settlementPeriod"):
    """
    FT-style two-row figure.

      Row 1: line + marker plot of forecast vs actual
      Row 2: table:
        settlementPeriod, forecast_MW, actual_MW, diff_MW
    """
    if df.empty:
        print(f"{fuel_label}: no data to plot.")
        return

    df = df.copy()

    if x_axis not in ("settlementPeriod", "startTime_cest"):
        raise ValueError("x_axis must be 'settlementPeriod' or 'startTime_cest'")

    # --- X values and ordering ---
    if x_axis == "settlementPeriod":
        # Local day order: 47, 48, 1..46
        order = settlement_period_order()
        df["settlementPeriod"] = df["settlementPeriod"].astype(int)
        df["settlementPeriod_str"] = df["settlementPeriod"].astype(str)

        order_index = {sp: i for i, sp in enumerate(order)}
        df["sp_sort_key"] = df["settlementPeriod_str"].map(order_index).fillna(len(order))

        df = df.sort_values("sp_sort_key").reset_index(drop=True)

        x_vals = df["settlementPeriod_str"]
        x_title = "Settlement Period"
        category_args = dict(categoryorder="array", categoryarray=order)
    else:
        df = df.sort_values("startTime_cest").reset_index(drop=True)
        x_vals = df["startTime_cest"]
        x_title = "Local start time"
        category_args = {}

    # Local date/time for title from startTime_cest (CE(S)T)
    local_dt = df["startTime_cest"].iloc[0]
    date_str = local_dt.strftime("%d %b %Y")
    tz_str = local_dt.strftime("%Z")
    title = f"{fuel_label} generation — forecast vs actual — {date_str} ({tz_str})"

    fig = make_subplots(
        rows=2,
        cols=1,
        shared_xaxes=False,
        vertical_spacing=0.08,
        row_heights=[0.65, 0.35],
        specs=[[{"type": "scatter"}],
               [{"type": "table"}]],
    )

    # Forecast line (FT red)
    fig.add_trace(
        go.Scatter(
            x=x_vals,
            y=df["forecast_MW"],
            mode="lines+markers",
            name=f"{fuel_label} forecast",
            marker=dict(size=7),
            line=dict(width=2, color=ft_red),
            hovertemplate=(
                f"{x_axis}: %{{x}}<br>"
                "Forecast: %{y:.1f} MW<extra></extra>"
            ),
        ),
        row=1, col=1,
    )

    # Actual line (FT green)
    fig.add_trace(
        go.Scatter(
            x=x_vals,
            y=df["actual_MW"],
            mode="lines+markers",
            name=f"{fuel_label} actual",
            marker=dict(size=7),
            line=dict(width=2, dash="dot", color=ft_green),
            hovertemplate=(
                f"{x_axis}: %{{x}}<br>"
                "Actual: %{y:.1f} MW<extra></extra>"
            ),
        ),
        row=1, col=1,
    )

    # Layout / FT styling
    fig.update_layout(
        title=title,
        paper_bgcolor=paper_bg,
        plot_bgcolor=plot_bg,
        font=dict(family="Georgia, serif", color=tick_col),
        legend_title_text="Series",
        hovermode="x unified" if x_axis == "settlementPeriod" else "closest",
        margin=dict(t=60, b=40),
    )

    fig.update_traces(
        selector=dict(type="scatter"),
        marker=dict(
            size=7,
            opacity=0.9,
            line=dict(width=0),
        ),
    )

    # Axes
    fig.update_yaxes(
        title_text="Generation (MW)",
        row=1, col=1,
        gridcolor=grid_col,
        zeroline=True,
        zerolinecolor=axis_col,
        zerolinewidth=2,
        linecolor=axis_col,
        tickfont=dict(color=tick_col),
    )

    fig.update_xaxes(
        row=1, col=1,
        showgrid=False,
        linecolor=axis_col,
        tickfont=dict(color=tick_col),
        **category_args,
    )

    fig.update_xaxes(
        row=2, col=1,
        title_text=x_title,
        showgrid=False,
        linecolor=axis_col,
        tickfont=dict(color=tick_col),
        **category_args,
    )

    # Table – same ordering as df
    table_df = df[["settlementPeriod", "forecast_MW", "actual_MW", "diff_MW"]].copy()

    table_df["forecast_MW"] = table_df["forecast_MW"].round(1)
    table_df["actual_MW"] = table_df["actual_MW"].round(1)
    table_df["diff_MW"] = table_df["diff_MW"].round(1)

    fig.add_trace(
        go.Table(
            header=dict(
                values=["SP", "Forecast (MW)", "Actual (MW)", "Actual - Forecast (MW)"],
                align="center",
                font=dict(size=12, color=paper_bg),
                fill_color=axis_col,
            ),
            cells=dict(
                values=[
                    table_df["settlementPeriod"],
                    table_df["forecast_MW"],
                    table_df["actual_MW"],
                    table_df["diff_MW"],
                ],
                align="center",
                fill_color=plot_bg,
                font=dict(color=tick_col),
            ),
            columnwidth=[0.8, 1.4, 1.4, 1.6],
        ),
        row=2, col=1,
    )

    base = f"forecast_vs_actual_{fuel_label.lower()}_{date_str.replace(' ', '_')}"

    try:
        fig.write_image(base + ".png")
    except Exception:
        pass

    fig.write_html(base + ".html", include_plotlyjs="cdn")

    fig.show()


In [ ]:
def print_forecast_error_summary(df, fuel_label="Wind"):
    """
    Simple stats for commentary.
    """
    if df.empty:
        print(f"{fuel_label}: no data.")
        return

    diffs = df["diff_MW"].astype(float)

    mean_err = diffs.mean()
    mae = diffs.abs().mean()
    worst_under = diffs.min()
    worst_over = diffs.max()

    print(f"{fuel_label} mean error (actual - forecast): {mean_err:.1f} MW")
    print(f"{fuel_label} mean absolute error:           {mae:.1f} MW")
    print(f"{fuel_label} max under-forecast:            {worst_under:.1f} MW")
    print(f"{fuel_label} max over-forecast:             {worst_over:.1f} MW")


In [ ]:
def run_part2_wind_solar(date, do_plots=True, x_axis="settlementPeriod"):
    """
    Fetch, align, plot, and summarise wind/solar forecast vs actuals
    for a local (Europe/Berlin) calendar day.

    Local day D (00:00–23:30) uses:
      - SP 47–48 from previous UTC settlementDate
      - SP 1–46 from the selected UTC settlementDate
    """
    print(f"Part 2 – wind & solar forecast vs actuals for local day {date}")

    # Previous day (for SP 47–48) and current day (SP 1–46)
    date_obj = dt.datetime.strptime(date, "%Y-%m-%d")
    prev_obj = date_obj - dt.timedelta(days=1)
    prev_str = prev_obj.strftime("%Y-%m-%d")

    # --- Forecasts: previous day (47–48) + current day (1–46) ---
    r_fore_prev = fetch_wind_solar_forecast(prev_str)
    r_fore_curr = fetch_wind_solar_forecast(date)

    df_fore_prev = forecast_req_to_df(r_fore_prev)
    df_fore_curr = forecast_req_to_df(r_fore_curr)

    df_fore_prev["settlementPeriod"] = df_fore_prev["settlementPeriod"].astype(int)
    df_fore_curr["settlementPeriod"] = df_fore_curr["settlementPeriod"].astype(int)

    df_fore_prev_sel = df_fore_prev[df_fore_prev["settlementPeriod"].isin([47, 48])]
    df_fore_curr_sel = df_fore_curr[df_fore_curr["settlementPeriod"].between(1, 46)]

    df_fore_local = pd.concat([df_fore_prev_sel, df_fore_curr_sel], ignore_index=True)

    # --- Actuals: previous day (47–48) + current day (1–46) ---
    r_act_prev = fetch_wind_solar_actuals(prev_str)
    r_act_curr = fetch_wind_solar_actuals(date)

    df_act_prev = actuals_req_to_df(r_act_prev)
    df_act_curr = actuals_req_to_df(r_act_curr)

    df_act_prev["settlementPeriod"] = df_act_prev["settlementPeriod"].astype(int)
    df_act_curr["settlementPeriod"] = df_act_curr["settlementPeriod"].astype(int)

    df_act_prev_sel = df_act_prev[df_act_prev["settlementPeriod"].isin([47, 48])]
    df_act_curr_sel = df_act_curr[df_act_curr["settlementPeriod"].between(1, 46)]

    df_act_local = pd.concat([df_act_prev_sel, df_act_curr_sel], ignore_index=True)

    print(f"Forecast rows (local day): {len(df_fore_local)}")
    print(f"Actual rows   (local day): {len(df_act_local)}")

    # Align, split, plot, summarise
    merged = prepare_wind_solar_merged(df_fore_local, df_act_local)
    df_wind, df_solar = split_wind_solar(merged)

    print(f"Wind rows (merged):  {len(df_wind)}")
    print(f"Solar rows (merged): {len(df_solar)}")

    if do_plots:
        plot_forecast_vs_actual_with_table(df_wind, fuel_label="Wind", x_axis=x_axis)
        plot_forecast_vs_actual_with_table(df_solar, fuel_label="Solar", x_axis=x_axis)

    print_forecast_error_summary(df_wind, fuel_label="Wind")
    print_forecast_error_summary(df_solar, fuel_label="Solar")

    return df_wind, df_solar


In [ ]:
df_wind, df_solar = run_part2_wind_solar(
    date="2025-11-11",
    do_plots=True,
    x_axis="settlementPeriod",
)


Part 2 – wind & solar forecast vs actuals for local day 2025-11-11
 Forecast attempt 1 ...
Forecast request OK.
 Forecast attempt 1 ...
Forecast request OK.
 Actuals attempt 1 ...
 Actuals request OK.
 Actuals attempt 1 ...
 Actuals request OK.
Forecast rows (local day): 144
Actual rows   (local day): 288
Wind rows (merged):  48
Solar rows (merged): 48


Wind mean error (actual - forecast): 681.7 MW
Wind mean absolute error:           1144.3 MW
Wind max under-forecast:            -1573.9 MW
Wind max over-forecast:             2842.1 MW
Solar mean error (actual - forecast): 8.1 MW
Solar mean absolute error:           31.3 MW
Solar max under-forecast:            -182.0 MW
Solar max over-forecast:             245.0 MW


In [ ]:
# =========================================================
#   FT-ish styling: colours, fonts, backgrounds
# =========================================================

import plotly.express as px  # already imported earlier, but safe to re-import

# FT-ish palette
paper_bg = "#f2e6d8"      # warm FT paper
plot_bg  = "#f2e6d8"      # same as paper for seamless look
grid_col = "#e3d5c6"      # very soft grid
axis_col = "#b0977b"      # axis line colour
tick_col = "#6b5a4b"      # text colour

ft_green = "#7bb274"      # muted soft green
ft_red   = "#c6665c"      # muted salmon red


In [ ]:
def plot_forecast_vs_actual_with_table(df, fuel_label="Wind", x_axis="settlementPeriod"):
    """
    FT-style two-row Plotly figure:

      Row 1: line + marker plot of forecast vs actual (FT colours)
      Row 2: table with:
        - settlementPeriod
        - forecast_MW
        - actual_MW
        - diff_MW (actual - forecast)

    Parameters
    ----------
    df : DataFrame
        Must contain:
          settlementDate, settlementPeriod, forecast_MW, actual_MW, diff_MW, startTime_cest.
    fuel_label : str
        "Wind" or "Solar" (used in titles and legend).
    x_axis : {"settlementPeriod", "startTime_cest"}
        X-axis choice. For this task the default is settlementPeriod.
    """
    if df.empty:
        print(f"⚠️ No data available for {fuel_label}.")
        return

    # Apply BMRS-style custom ordering
    order = settlement_period_order()
    df = df.copy()
    df["settlementPeriod_str"] = df["settlementPeriod"].astype(str)

    # Sort according to custom category order
    df["sp_sort_key"] = df["settlementPeriod_str"].apply(lambda s: order.index(s))
    df = df.sort_values("sp_sort_key").reset_index(drop=True)


    if x_axis not in ("settlementPeriod", "startTime_cest"):
        raise ValueError("x_axis must be 'settlementPeriod' or 'startTime_cest'")

    x_col = x_axis

    # Assume a single settlementDate in df
    date_val = pd.to_datetime(df["settlementDate"].iloc[0])
    date_str = date_val.strftime("%d %b %Y")

    title = f"{fuel_label} generation — forecast vs actual — {date_str} (CEST)"

    # Subplots: top = scatter, bottom = table
    fig = make_subplots(
        rows=2,
        cols=1,
        shared_xaxes=False,
        vertical_spacing=0.08,
        row_heights=[0.65, 0.35],
        specs=[[{"type": "scatter"}],
               [{"type": "table"}]],
    )

    # ---------- Row 1: forecast line (FT red) ----------
    fig.add_trace(
        go.Scatter(
            x=df["settlementPeriod_str"] if x_axis=="settlementPeriod" else df[x_col],
            y=df["forecast_MW"],
            mode="lines+markers",
            name=f"{fuel_label} forecast",
            marker=dict(size=7),
            line=dict(width=2, color=ft_red),
            hovertemplate=(
                f"{x_col}: %{{x}}<br>"
                "Forecast: %{y:.1f} MW<extra></extra>"
            ),
        ),
        row=1, col=1,
    )

    # ---------- Row 1: actual line (FT green, dotted) ----------
    fig.add_trace(
        go.Scatter(
            x=df["settlementPeriod_str"] if x_axis=="settlementPeriod" else df[x_col],
            y=df["actual_MW"],
            mode="lines+markers",
            name=f"{fuel_label} actual",
            marker=dict(size=7),
            line=dict(width=2, dash="dot", color=ft_green),
            hovertemplate=(
                f"{x_col}: %{{x}}<br>"
                "Actual: %{y:.1f} MW<extra></extra>"
            ),
        ),
        row=1, col=1,
    )

    # Axis labels
    if x_axis == "settlementPeriod":
        x_title = "Settlement Period"
    else:
        x_title = "Local start time (CEST)"

    # --------- FT-style layout: background, fonts ----------
    fig.update_layout(
        title=title,
        paper_bgcolor=paper_bg,
        plot_bgcolor=plot_bg,
        font=dict(family="Georgia, serif", color=tick_col),
        legend_title_text="Series",
        hovermode="x unified" if x_axis == "settlementPeriod" else "closest",
        margin=dict(t=60, b=40),
    )

    # Apply FT marker style only to scatter traces
    fig.update_traces(
        selector=dict(type="scatter"),
        marker=dict(
            size=7,
            opacity=0.9,
            line=dict(width=0)
        )
    )

    # Axes & grid – FT-ish
    fig.update_yaxes(
        title_text="Generation (MW)",
        row=1, col=1,
        gridcolor=grid_col,
        zerolinecolor=axis_col,
        linecolor=axis_col,
        tickfont=dict(color=tick_col),
    )

    fig.update_xaxes(
        title_text=x_title,
        row=2, col=1,   # x-axis title under the table (overall)
        showgrid=False,
        linecolor=axis_col,
        tickfont=dict(color=tick_col),
    )

    # ---------- Row 2: FT-style table ----------
    table_df = df[["settlementPeriod", "forecast_MW", "actual_MW", "diff_MW"]].copy()
    table_df["forecast_MW"] = table_df["forecast_MW"].round(1)
    table_df["actual_MW"] = table_df["actual_MW"].round(1)
    table_df["diff_MW"] = table_df["diff_MW"].round(1)

    fig.add_trace(
        go.Table(
            header=dict(
                values=["SP", "Forecast (MW)", "Actual (MW)", "Actual - Forecast (MW)"],
                align="center",
                font=dict(size=12, color=paper_bg),
                fill_color=axis_col,
            ),
            cells=dict(
                values=[
                    table_df["settlementPeriod_str"],
                    table_df["forecast_MW"],
                    table_df["actual_MW"],
                    table_df["diff_MW"],
                ],
                align="center",
                fill_color=plot_bg,
                font=dict(color=tick_col),
            ),
            columnwidth=[0.8, 1.4, 1.4, 1.6],
        ),
        row=2, col=1,
    )
    if x_axis == "settlementPeriod":
        fig.update_xaxes(
        categoryorder="array",
        categoryarray=order,
        title_text=x_title,
        row=2, col=1,
        showgrid=False,
        linecolor=axis_col,
        tickfont=dict(color=tick_col),
    )


    fig.show()


In [ ]:
df_wind, df_solar = run_part2_wind_solar(
    date="2025-11-10",
    do_plots=True,
    x_axis="settlementPeriod",
)


Part 2 – wind & solar forecast vs actuals for local day 2025-11-10
 Forecast attempt 1 ...
Forecast request OK.
 Forecast attempt 1 ...
Forecast request OK.
 Actuals attempt 1 ...
 Actuals request OK.
 Actuals attempt 1 ...
 Actuals request OK.
Forecast rows (local day): 144
Actual rows   (local day): 288
Wind rows (merged):  48
Solar rows (merged): 48


KeyError: 'settlementPeriod_str'